<a href="https://colab.research.google.com/github/brunananan/-1--42-_-_lab3-4-added/blob/main/%D0%A1%D1%8B%20%D0%AD%D0%BD%D1%8C%D1%86%D0%B7%D1%8Dserda_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

作业目的
- 处理心脏数据集以解决二元分类问题

关键技能
- 应用 autoML 模型，计算分类器的准确性

任务：
从数据集中加载 5000 行数据：（参见名为 “模块 3....” 的文件）：https://github.com/AI-is-out-there/data2lab.git。从下载的数据集中生成一个训练样本，包括
列： ['Count_subj'、'rr_interval'、'p_end'、'qrs_onset'、'qrs_end'、'p_axis'、'qrs_axis'、't_axis'、
健康状况']。
数据集包括数字心电图参数和分类特征 Healthy_Status。
探索 AutoML 框架，证明选择最佳框架的合理性。将其用于二元
分类。构建误差矩阵（混淆矩阵）并计算 F1 指标，以评估基于
根据心电图参数数据对 Healthy_Status 特征训练分类器。

链接到基准解决方案：
https://github.com/TAUforPython/BioMedAI/blob/main/ML%20ECG%20classification.ipynb

In [ ]:
# @title 1. 安装所需的库 (Install necessary libraries)
# 安装 H2O (用于 AutoML) 和 GitPython (用于克隆仓库)
!pip install h2o gitpython
!pip install pycaret[full]

In [ ]:
import pandas as pd

# GitHub 仓库中原始文件的正确链接
file_url = "https://raw.githubusercontent.com/AI-is-out-there/data2lab/main/%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C%203%20-%20%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20-%20%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0.csv"

# 2.需要选择的列列表
selected_columns = ['Count_subj', 'rr_interval', 'p_end', 'qrs_onset', 'qrs_end', 'p_axis', 'qrs_axis', 't_axis', 'Healthy_Status']

# 1.加载前 5000 行
try:
    df = pd.read_csv(file_url, nrows=5000)

    # 选择必要的列
    df = df[selected_columns]

    print("数据集加载和准备成功。")
    print(f"加载的行数: {len(df)}")
    print("数据集前 5 行:")
    print(df.head())
    print("\n数据集信息:")
    df.info()

except Exception as e:
    print(f"加载或处理数据时出错: {e}")
    # 如果加载出错，停止执行后续代码
    df = None

数据集加载和准备成功。
加载的行数: 5000
数据集前 5 行:
   Count_subj  rr_interval  p_end  qrs_onset  qrs_end  p_axis  qrs_axis  \
0          27          659    128        170      258      81        77   
1          93          722    124        162      246      77        75   
2           3          600    130        162      244      79        72   
3           7          659    146        180      254      79        66   
4           2          659  29999        504      590      84        80   

   t_axis  Healthy_Status  
0      79               0  
1      70               0  
2      77               1  
3      69               1  
4      77               0  

数据集信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Count_subj      5000 non-null   int64
 1   rr_interval     5000 non-null   int64
 2   p_end           5000 non-null   int64
 3   qrs_onset    

In [ ]:
# 检查数据集是否成功加载
if df is not None:
    # 确保已安装 PyCaret (如果尚未安装: pip install pycaret[full])
    from pycaret.classification import *

    # 初始化 PyCaret 环境
    # target='Healthy_Status' - 指定目标变量
    # session_id - 用于结果的复现性
    # normalize = True - 对特征进行归一化
    # fix_imbalance = True - 尝试纠正类别不平衡
    try:
        # PyCaret 会自动将数据分割为训练集和测试集（默认为 70/30）
        clf1 = setup(data = df,
                     target = 'Healthy_Status',
                     session_id = 123,
                     normalize = True, # 对特征进行归一化
                     fix_imbalance = True # 尝试纠正类别不平衡
                    )

        # 比较模型
        # sort='F1' - 按照 F1 指标排序
        print("\n使用 PyCaret 比较模型 (按 F1 排序)...")
        best_model = compare_models(sort='F1')

        print(f"\n根据 F1 指标选出的最佳模型: {type(best_model).__name__}")

        # 在保留 (holdout) 测试集上评估最佳模型
        # predict_model 会自动输出保留数据集的混淆矩阵及其他指标
        print("\n在保留测试集上评估最佳模型:")
        evaluation_results = predict_model(best_model)

        # predict_model 已经输出了所有指标，包括 F1 和 Confusion Matrix。
        # 输出中会包含如下信息：
        # - 各项指标（Accuracy, AUC, F1 等）在测试集上的值
        # - 混淆矩阵 (Confusion Matrix)

        print("\n任务完成。")

    except Exception as e:
        print(f"\n使用 PyCaret 时出错: {e}")

else:
    print("\n数据加载失败，无法执行分类。")

,Description,Value
0,Session id,123
1,Target,Healthy_Status
2,Target type,Binary
3,Original data shape,"(5000, 9)"
4,Transformed data shape,"(7156, 9)"
5,Transformed train set shape,"(5656, 9)"
6,Transformed test set shape,"(1500, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple



使用 PyCaret 比较模型 (按 F1 排序)...


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8549,0.9189,0.8067,0.5897,0.6808,0.5899,0.6024,6.5710
gbc,Gradient Boosting Classifier,0.8400,0.9139,0.8558,0.5546,0.6726,0.5731,0.5971,0.9840
et,Extra Trees Classifier,0.8483,0.9141,0.7858,0.5770,0.6651,0.5700,0.5816,0.5590
ada,Ada Boost Classifier,0.8320,0.9136,0.8677,0.5399,0.6650,0.5611,0.5897,0.4210
rf,Random Forest Classifier,0.8517,0.9174,0.7680,0.5873,0.6649,0.5719,0.5810,1.2470
lightgbm,Light Gradient Boosting Machine,0.8486,0.9151,0.7412,0.5838,0.6525,0.5576,0.5646,0.6940
xgboost,Extreme Gradient Boosting,0.8460,0.9098,0.7144,0.5804,0.6402,0.5436,0.5486,0.1810
dt,Decision Tree Classifier,0.8343,0.7681,0.6607,0.5598,0.6048,0.5011,0.5048,0.0970
svm,SVM - Linear Kernel,0.6234,0.8073,0.9717,0.3375,0.4996,0.2997,0.4092,0.0590
lr,Logistic Regression,0.6094,0.8026,0.9926,0.3298,0.4948,0.2900,0.4083,0.8810


Processing:   0%|          | 0/69 [00:00<?, ?it/s]


根据 F1 指标选出的最佳模型: CatBoostClassifier

在保留测试集上评估最佳模型:


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.8480,0.9135,0.7882,0.5761,0.6657,0.5704,0.5821



任务完成。
